# Discrete event simulation
https://docs.netsquid.org/latest-release/tutorial.pydynaa.html

## A quantum ping pong example

In [1]:
import netsquid as ns
import pydynaa
ns.set_random_state(seed=42)

In [2]:
class PingEntity(pydynaa.Entity):
    ping_evtype = pydynaa.EventType('PING_EVENT', 'A ping event')
    delay = 10.

    def start(self, qubit):
        # start the game by scheduling the first ping event after delay
        self.qubit = qubit
        self._schedule_now(PingEntity.ping_evtype)

    def wait_for_pong(self, pong_entity):
        # setup this entity to listen for pong events from a PongEntity
        pong_handler = pydynaa.EventHandler(self._handle_pong_event)
        self._wait(pong_handler, entity=pong_entity, event_type=PongEntity.pong_evtype)

    def _handle_pong_event(self, event):
        # callback function called by the pong handler when pong event is triggered
        m, prob = ns.qubits.measure(self.qubit, observable=ns.Z)
        labels_z = ('|0>', '|1>')
        print(f'{ns.sim_time():.1f}: Pong event! PingEntity measured {labels_z[m]} with probability {prob:.2f}')
        self._schedule_after(PingEntity.delay, PingEntity.ping_evtype)


class PongEntity(pydynaa.Entity):
    pong_evtype = pydynaa.EventType('PONG_EVENT', 'A pong event')
    delay = 10.

    def wait_for_ping(self, ping_entity):
        # setup this entity to listen for ping events from a PingEntity
        ping_handler = pydynaa.EventHandler(self._handle_ping_event)
        self._wait(ping_handler, entity=ping_entity, event_type=PingEntity.ping_evtype)

    def _handle_ping_event(self, event):
        # callback function called by the ping handler when ping event is triggered
        m, prob = ns.qubits.measure(event.source.qubit, observable=ns.X)
        labels_x = ('|+>', '|->')
        print(f"{ns.sim_time():.1f}: Ping event! PongEntity measured {labels_x[m]} with probability {prob:.2f}")
        self._schedule_after(PongEntity.delay, PongEntity.pong_evtype)


In [3]:
# create entities and register them to each other
ping = PingEntity()
pong = PongEntity()
ping.wait_for_pong(pong)
pong.wait_for_ping(ping)

qubit, = ns.qubits.create_qubits(1)
ping.start(qubit)

In [4]:
stats = ns.sim_run(end_time=200)

0.0: Ping event! PongEntity measured |+> with probability 0.50
10.0: Pong event! PingEntity measured |1> with probability 0.50
20.0: Ping event! PongEntity measured |-> with probability 0.50
30.0: Pong event! PingEntity measured |1> with probability 0.50
40.0: Ping event! PongEntity measured |+> with probability 0.50
50.0: Pong event! PingEntity measured |0> with probability 0.50
60.0: Ping event! PongEntity measured |+> with probability 0.50
70.0: Pong event! PingEntity measured |1> with probability 0.50
80.0: Ping event! PongEntity measured |-> with probability 0.50
90.0: Pong event! PingEntity measured |1> with probability 0.50
100.0: Ping event! PongEntity measured |+> with probability 0.50
110.0: Pong event! PingEntity measured |1> with probability 0.50
120.0: Ping event! PongEntity measured |-> with probability 0.50
130.0: Pong event! PingEntity measured |0> with probability 0.50
140.0: Ping event! PongEntity measured |+> with probability 0.50
150.0: Pong event! PingEntity measur

In [6]:
print(stats)


Simulation summary

Elapsed wallclock time: 0:00:00.005043
Elapsed simulation time: 2.00e+02 [ns]
Triggered events: 20
Handled callbacks: 20
Total quantum operations: 20
Frequent quantum operations: MEASURE = 20
Max qstate size: 1 qubits
Mean qstate size: 1.00 qubits



## Event expressions by example: quantum teleportation